In [72]:
from pyspark.sql import SparkSession
from datetime import datetime
import json
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [1]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
df = spark.read.json('sample_10e3.json')
df.show(5)

+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|[5cf83b752f3843a0...|       0|         21|            66|            8|              90|        219|[2019-06-01T00:00...|e4d55deb9ac172a8d...|        1|        |
|[5cf83b762f3843a0...|       4|         19|           136|           19|              71|        359|[2019-06-01T00:00...|8a0c4123e924a50a9...|        1|   28039|
|[5cf83b762f3843a0...|       4|         17|            38|            7|              39|        375|[2019-06-01T00:00...|a6a9c1f74a6849600...|        1|   28013|
|[5cf83b762f3843a0...|

In [7]:
def get_data(line):
    data = json.loads(line)
    data["_id"] = data["_id"]["$oid"]
    data['unplug_hourTime'] = datetime.strptime(data['unplug_hourTime']['$date'], "%Y-%m-%dT%H:%M:%S.%f%z")
    return data

In [8]:
rdd = sc.textFile('sample_10e2.json').map(get_data)
df = spark.createDataFrame(rdd)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/session.py:401: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [9]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- ageRange: long (nullable = true)
 |-- idplug_base: long (nullable = true)
 |-- idplug_station: long (nullable = true)
 |-- idunplug_base: long (nullable = true)
 |-- idunplug_station: long (nullable = true)
 |-- travel_time: long (nullable = true)
 |-- unplug_hourTime: timestamp (nullable = true)
 |-- user_day_code: string (nullable = true)
 |-- user_type: long (nullable = true)
 |-- zip_code: string (nullable = true)



In [10]:
df.show(5)

+--------------------+--------+-----------+--------------+-------------+----------------+-----------+-------------------+--------------------+---------+--------+
|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|    unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+-------------------+--------------------+---------+--------+
|5cf83b752f3843a01...|       0|         21|            66|            8|              90|        219|2019-06-01 00:00:00|e4d55deb9ac172a8d...|        1|        |
|5cf83b762f3843a01...|       4|         19|           136|           19|              71|        359|2019-06-01 00:00:00|8a0c4123e924a50a9...|        1|   28039|
|5cf83b762f3843a01...|       4|         17|            38|            7|              39|        375|2019-06-01 00:00:00|a6a9c1f74a6849600...|        1|   28013|
|5cf83b762f3843a01...|      

In [17]:
df.select('ageRange').show(5)

+--------+
|ageRange|
+--------+
|       0|
|       4|
|       4|
|       5|
|       4|
+--------+
only showing top 5 rows



In [19]:
df.groupBy('user_type').count().show()

+---------+-----+
|user_type|count|
+---------+-----+
|        1|  100|
+---------+-----+



In [30]:
df = df.drop('_id','user_day_code')
df.show(5)

+--------+-----------+--------------+-------------+----------------+-----------+-------------------+---------+--------+
|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|    unplug_hourTime|user_type|zip_code|
+--------+-----------+--------------+-------------+----------------+-----------+-------------------+---------+--------+
|       0|         21|            66|            8|              90|        219|2019-06-01 00:00:00|        1|        |
|       4|         19|           136|           19|              71|        359|2019-06-01 00:00:00|        1|   28039|
|       4|         17|            38|            7|              39|        375|2019-06-01 00:00:00|        1|   28013|
|       5|          4|            90|           21|              66|        264|2019-06-01 00:00:00|        1|   28009|
|       4|          3|           166|           13|             152|        367|2019-06-01 00:00:00|        1|   28006|
+--------+-----------+--------------+---

In [32]:
df.select('unplug_hourTime').show(10)

+-------------------+
|    unplug_hourTime|
+-------------------+
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
|2019-06-01 00:00:00|
+-------------------+
only showing top 10 rows



In [33]:
df.groupBy('unplug_hourTime').count().show()

+-------------------+-----+
|    unplug_hourTime|count|
+-------------------+-----+
|2019-06-01 00:00:00|  100|
+-------------------+-----+



In [34]:
df = df.drop('unplug_hourTime')
df.show(5)

+--------+-----------+--------------+-------------+----------------+-----------+---------+--------+
|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|user_type|zip_code|
+--------+-----------+--------------+-------------+----------------+-----------+---------+--------+
|       0|         21|            66|            8|              90|        219|        1|        |
|       4|         19|           136|           19|              71|        359|        1|   28039|
|       4|         17|            38|            7|              39|        375|        1|   28013|
|       5|          4|            90|           21|              66|        264|        1|   28009|
|       4|          3|           166|           13|             152|        367|        1|   28006|
+--------+-----------+--------------+-------------+----------------+-----------+---------+--------+
only showing top 5 rows



In [48]:
df = df.where(df.user_type == 1)

In [49]:
df = df.drop('user_type')
df.show(5)

+--------+-----------+--------------+-------------+----------------+-----------+--------+
|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|zip_code|
+--------+-----------+--------------+-------------+----------------+-----------+--------+
|       0|         21|            66|            8|              90|        219|        |
|       4|         19|           136|           19|              71|        359|   28039|
|       4|         17|            38|            7|              39|        375|   28013|
|       5|          4|            90|           21|              66|        264|   28009|
|       4|          3|           166|           13|             152|        367|   28006|
+--------+-----------+--------------+-------------+----------------+-----------+--------+
only showing top 5 rows



In [53]:
df.groupBy('zip_code').count().show(5)

+--------+-----+
|zip_code|count|
+--------+-----+
|   28039|    1|
|   28019|    1|
|   28015|    2|
|   28001|    1|
|   28030|    1|
+--------+-----+
only showing top 5 rows



In [57]:
df.select('zip_code').distinct().count()

31

In [58]:
df = df.drop('zip_code')
df.show(5)

+--------+-----------+--------------+-------------+----------------+-----------+
|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|
+--------+-----------+--------------+-------------+----------------+-----------+
|       0|         21|            66|            8|              90|        219|
|       4|         19|           136|           19|              71|        359|
|       4|         17|            38|            7|              39|        375|
|       5|          4|            90|           21|              66|        264|
|       4|          3|           166|           13|             152|        367|
+--------+-----------+--------------+-------------+----------------+-----------+
only showing top 5 rows



In [69]:
df.columns

['ageRange',
 'idplug_base',
 'idplug_station',
 'idunplug_base',
 'idunplug_station',
 'travel_time']

In [73]:
columnJoin = VectorAssembler(inputCols = ['ageRange','idplug_base','idplug_station','idunplug_base','idunplug_station'],outputCol = 'features')
t_df = columnJoin.transform(df)
t_df = t_df.select(['features','travel_time'])
t_df.show(5)

+--------------------+-----------+
|            features|travel_time|
+--------------------+-----------+
|[0.0,21.0,66.0,8....|        219|
|[4.0,19.0,136.0,1...|        359|
|[4.0,17.0,38.0,7....|        375|
|[5.0,4.0,90.0,21....|        264|
|[4.0,3.0,166.0,13...|        367|
+--------------------+-----------+
only showing top 5 rows



In [83]:
t_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- travel_time: long (nullable = true)



In [74]:
splits = t_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [90]:
lr = LinearRegression(featuresCol = 'features', labelCol='travel_time')
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [9.799760781641279,3.755370862660791,0.30123123361074516,2.7128952920364893,0.827196508219447]
Intercept: 386.5191902222934


In [91]:
summ = lr_model.summary

In [93]:
print(summ.rootMeanSquaredError)
print(summ.r2)

241.2767730909206
0.058319041033486174
